#### Dataset:
    1. Commercial Flight US - January 2024
    2. City Code
    3. Airlines Code (IATA)

In [ ]:
import pandas as pd
import numpy as np

data1 = "../data/raw/T_ONTIME_MARKETING.csv"
data2 = "../data/raw/Consumer_Airfare_Report__Table_2_-_Top_1_000_City-Pair_Markets_20240827.csv"
data3 = "../data/raw/airlines.dat"

df1 = pd.read_csv(data1, low_memory=False)
df2 = pd.read_csv(data2)
df3 = pd.read_csv(data3, encoding='utf8')
df3.columns = ["airline_id","name","alias","iata","icao","callsign","country","active"]

#Cleaning flight table (Primary key: 'FL_DATE', 'OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN_CITY_MARKET_ID','DEP_TIME')
flight = df1[['FL_DATE','OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN_CITY_MARKET_ID','DEST_CITY_MARKET_ID',
              'CRS_DEP_TIME','DEP_TIME', 'DEP_DELAY', 'ARR_TIME', 'ARR_DELAY','ARR_DELAY_NEW',
           'CANCELLED','DISTANCE']]
flight['FL_DATE'] = pd.to_datetime(flight['FL_DATE'])
flight['FL_DATE'] = flight['FL_DATE'].dt.strftime('%Y%m%d')

def convert_to_hhmmss(time_float):
    try: # Zero-pad the string to ensure it is at least 4 digits long
        time_str = str(int(time_float)).zfill(4)
        # Return the formatted time string as hhmmss
        return f"{time_str[:2]}{time_str[2:4]}00"
    except ValueError:
        return None

# Apply the conversion function to the time_column
flight['CRS_DEP_TIME'] = flight['DEP_TIME'].apply(convert_to_hhmmss)
flight['DEP_TIME'] = flight['DEP_TIME'].apply(convert_to_hhmmss)
flight['ARR_TIME'] = flight['ARR_TIME'].apply(convert_to_hhmmss)

# Cleaning city table (Primary key: citymarketid)
city = df2[(df2["Year"]==2024)].reset_index()
city = city[['citymarketid', 'city','markets','cur_passengers','cur_fare']]
city.columns = ['citymarketid', 'city_name','markets','cur_passengers','cur_fare']
city["state"]=city['city_name'].str.split(', ').str[1].str[:2]
city["city_name"]=city['city_name'].str.split(', ').str[0]
city = city[['citymarketid', 'city_name', 'state','markets','cur_passengers','cur_fare']]

# Cleaning airlines table (Primary key: iata)
airlines = df3.drop_duplicates(subset=['iata'])
airlines = airlines.drop(0)
airlines = airlines.drop(1).reset_index()
airlines = airlines[['name','iata','country']]
#Only keep 21 iata
df1["citymarketid"]=df1["ORIGIN_CITY_MARKET_ID"]
flight2 = flight.rename(columns= {'OP_UNIQUE_CARRIER':'iata'})
flight2 = flight2.drop_duplicates(subset=['iata'])
merge2 = pd.merge(airlines, flight2, on='iata', how='inner')
airlines = merge2[["name","iata","country"]]

In [ ]:
# Export cleaned tables as csv to be loaded to MySQL

#city
city.to_csv('../data/clean/city.csv', index=False)  

#flight
flight.to_csv('../data/clean/flight.csv', header=False, index=False)  

#airlines
airlines.to_csv('../data/clean/airlines.csv', index=False)  